In [53]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

means = [[-1, -1], [1, -1], [0, 1]]
cov = [[1, 0], [0, 1]]
N = 20
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X2 = np.random.multivariate_normal(means[2], cov, N)
X = np.concatenate((X0, X1, X2), axis=0)
y = np.asarray([0] * N + [1] * N + [2] * N)

scaler = StandardScaler()
X = scaler.fit_transform(X)

clf = MLPClassifier(solver='lbfgs', alpha=0.01, hidden_layer_sizes=(50,), max_iter=10000)
clf.fit(X, y)

y_pred = clf.predict(X)
acc = 100 * np.mean(y_pred == y)
print('Training accuracy: %.2f %%' % acc)


def softmax_stable(Z):
    """
    Compute softmax values for each sets of scores in Z.
    each ROW of Z is a set of scores.
    """
    e_Z = np.exp(Z - np.max(Z, axis = 1, keepdims = True))
    A = e_Z / e_Z.sum(axis = 1, keepdims = True)
    return A
def crossentropy_loss(Yhat, y):
    """
    Yhat: a numpy array of shape (Npoints, nClasses) -- predicted output
    y: a numpy array of shape (Npoints) -- ground truth.
    NOTE: We don’t need to use the one-hot vector here since most of elements
    are zeros. When programming in numpy, in each row of Yhat, we need to access
    to the corresponding index only.
    """
    id0 = range(Yhat.shape[0])
    return -np.mean(np.log(Yhat[id0, y]))
def mlp_init(d0, d1, d2):
    """
    Initialize W1, b1, W2, b2
    d0: dimension of input data
    d1: number of hidden unit
    d2: number of output unit = number of classes
    """
    W1 = 0.01*np.random.randn(d0, d1)
    b1 = np.zeros(d1)
    W2 = 0.01*np.random.randn(d1, d2)
    b2 = np.zeros(d2)
    return (W1, b1, W2, b2)
def mlp_predict(X, W1, b1, W2, b2):
    """
    Suppose that the network has been trained, predict class of new points.
    X: data matrix, each ROW is one data point.
    W1, b1, W2, b2: learned weight matrices and biases
    """
    Z1 = X.dot(W1) + b1 # shape (N, d1)
    A1 = np.maximum(Z1, 0) # shape (N, d1)
    Z2 = A1.dot(W2) + b2 # shape (N, d2)
    return np.argmax(Z2, axis=1)

Training accuracy: 100.00 %


In [17]:
def mlp_fit(X, y, W1, b1, W2, b2, eta):
    loss_hist = []
    for i in range(20000): # number of epoches
        # feedforward
        Z1 = X.dot(W1) + b1 # shape (N, d1)
        A1 = np.maximum(Z1, 0) # shape (N, d1)
        Z2 = A1.dot(W2) + b2 # shape (N, d2)
        Yhat = softmax_stable(Z2) # shape (N, d2)
        if i %1000 == 0: # print loss after each 1000 iterations
            loss = crossentropy_loss(Yhat, y)
            print("iter %d, loss: %f" %(i, loss))
            loss_hist.append(loss)
            
        # back propagation
        id0 = range(Yhat.shape[0])
        Yhat[id0, y] -=1
        E2 = Yhat/N # shape (N, d2)
        dW2 = np.dot(A1.T, E2) # shape (d1, d2)
        db2 = np.sum(E2, axis = 0) # shape (d2,)
        E1 = np.dot(E2, W2.T) # shape (N, d1)
        E1[Z1 <= 0] = 0 # gradient of ReLU, shape (N, d1)
        dW1 = np.dot(X.T, E1) # shape (d0, d1)
        db1 = np.sum(E1, axis = 0) # shape (d1,)
        # Gradient Descent update
        W1 += -eta*dW1
        b1 += -eta*db1
        W2 += -eta*dW2
        b2 += -eta*db2
    return (W1, b1, W2, b2, loss_hist)

In [19]:
# suppose X, y are training input and output, respectively
d0 = 2 # data dimension
d1 = h = 100 # number of hidden units
d2 = C = 3 # number of classes
eta = 1 # learning rate
(W1, b1, W2, b2) = mlp_init(d0, d1, d2)
(W1, b1, W2, b2, loss_hist) =mlp_fit(X, y, W1, b1, W2, b2, eta)
y_pred = mlp_predict(X, W1, b1, W2, b2)
acc = 100*np.mean(y_pred == y)
print('training accuracy: %.2f %%' % acc)


iter 0, loss: 1.098734
iter 1000, loss: 0.224423
iter 2000, loss: 0.116419
iter 3000, loss: 0.085216
iter 4000, loss: 0.035879
iter 5000, loss: 0.031397
iter 6000, loss: 0.030207
iter 7000, loss: 0.029803
iter 8000, loss: 0.029445
iter 9000, loss: 0.029299
iter 10000, loss: 0.029364
iter 11000, loss: 0.029017
iter 12000, loss: 0.028937
iter 13000, loss: 0.028861
iter 14000, loss: 0.028783
iter 15000, loss: 0.028710
iter 16000, loss: 0.029833
iter 17000, loss: 0.028990
iter 18000, loss: 0.028539
iter 19000, loss: 0.032915
training accuracy: 98.33 %
